In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import os
from time import time
import datetime

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

from tqdm.notebook import tqdm


In [2]:
def create_folds(num_starts, num_splits):
    
    folds = []
    
    # LOAD FILES
    train_feats = pd.read_csv('../input/lish-moa/train_features.csv')
    scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
    drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
    scored = scored.loc[train_feats['cp_type'] == 'trt_cp', :]
    drug = drug.loc[train_feats['cp_type'] == 'trt_cp', :]
    targets = scored.columns[1:]
    scored = scored.merge(drug, on = 'sig_id', how = 'left') 

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc <= 18].index.sort_values()
    vc2 = vc.loc[vc > 18].index.sort_values()
    
    for seed in range(num_starts):

        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}; dct2 = {}
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[targets])):
            dd = {k:fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop = True)
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[targets])):
            dd = {k:fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        scored['fold'] = scored.drug_id.map(dct1)
        scored.loc[scored.fold.isna(),'fold'] =\
            scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
        scored.fold = scored.fold.astype('int8')
        folds.append(scored.fold.values)
        
        del scored['fold']
        
    return np.stack(folds)

# Data Preparation

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

ss_krr = pd.read_csv('../input/lish-moa/sample_submission.csv')
ss_lr = ss_krr.copy()

cols = [c for c in ss_krr.columns.values if c != 'sig_id']
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [4]:
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [5]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 0.5, 72: 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    return - np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip))

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

In [6]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(n_quantiles = 100, output_distribution = 'normal', random_state = 42)
qt.fit(pd.concat([pd.DataFrame(train[GENES+CELLS]), pd.DataFrame(test[GENES+CELLS])]))
train[GENES+CELLS] = qt.transform(train[GENES+CELLS])
test[GENES+CELLS] = qt.transform(test[GENES+CELLS])

In [7]:
from sklearn.decomposition import PCA

# GENES
n_comp_genes = 550

data = pd.concat([pd.DataFrame(train[GENES]), pd.DataFrame(test[GENES])])
pca_genes = PCA(n_components=n_comp_genes, random_state = 42)
data2 = pca_genes.fit_transform(data[GENES])
train2 = data2[:train.shape[0]]; test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_genes)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_genes)])

train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

#CELLS
n_comp_cells = 70

data = pd.concat([pd.DataFrame(train[CELLS]), pd.DataFrame(test[CELLS])])
pca_cells = PCA(n_components=n_comp_cells, random_state = 42)
data2 = pca_cells.fit_transform(data[CELLS])
train2 = data2[:train.shape[0]]; test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_cells)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_cells)])

train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

In [8]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.8)
data = train.append(test)
data_transformed = var_thresh.fit_transform(data.iloc[:, 3:])

train_transformed = data_transformed[ : train.shape[0]]
test_transformed = data_transformed[-test.shape[0] : ]

train = pd.DataFrame(train[['cp_type','cp_time','cp_dose']].values.reshape(-1, 3),\
            columns=['cp_type','cp_time','cp_dose'])

train = pd.concat([train, pd.DataFrame(train_transformed)], axis=1)

test = pd.DataFrame(test[['cp_type','cp_time','cp_dose']].values.reshape(-1, 3),\
            columns=['cp_type','cp_time','cp_dose'])

test = pd.concat([test, pd.DataFrame(test_transformed)], axis=1)

print(train.shape)
print(test.shape)

(23814, 1040)
(3982, 1040)


In [9]:
train_targets = train_targets.loc[train['cp_type'] == 0].reset_index(drop = True)
train = train.loc[train['cp_type'] == 0].reset_index(drop = True)

print(train.shape)

(21948, 1040)


In [10]:
top_feats = np.arange(1, train.shape[1])
print(top_feats)

[   1    2    3 ... 1037 1038 1039]


In [11]:
N_SPLITS = 5
LBS = 0.0008
folds = create_folds(1, N_SPLITS)
print(folds)

[[1 4 2 ... 1 0 4]]


# Kernel Logistic Regression

In [12]:
res_krr = train_targets.copy()
ss_krr.loc[:, train_targets.columns] = 0
res_krr.loc[:, train_targets.columns] = 0

# for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits = N_SPLITS, random_state = 0, shuffle = True).split(train_targets, train_targets)):
for n, foldno in enumerate(set(folds[0])):
    start_time = time()
    tr = folds[0] != foldno
    te = folds[0] == foldno

    x_tr, x_val = train.values[tr][:, top_feats], train.values[te][:, top_feats]
    y_tr, y_val = train_targets.astype(float).values[tr], train_targets.astype(float).values[te]
    x_tt = test.values[:, top_feats]
    
    model = KernelRidge(alpha = 1, kernel = 'rbf')
    model.fit(x_tr, y_tr)

    ss_krr.loc[:, train_targets.columns] += model.predict(x_tt) / N_SPLITS
    fold_pred = model.predict(x_val)
    res_krr.loc[te, train_targets.columns] += fold_pred
    fold_score = log_loss_metric(train_targets.loc[te].values, fold_pred)
    print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] KRR: Fold {n}:', fold_score)

[01:05] KRR: Fold 0: 0.04252455401003149
[01:03] KRR: Fold 1: 0.03841322293093819
[01:03] KRR: Fold 2: 0.04263732823652618
[01:03] KRR: Fold 3: 0.040042855817107925
[01:04] KRR: Fold 4: 0.04423631630108548


In [13]:
print(f'Model OOF Metric: {log_loss_metric(train_targets.values, res_krr.values)}')

Model OOF Metric: 0.04157830500657903


# Platt Scaling

Train a Logistic Regression model to calibrate the results of Kernel Ridge Regression.

In [14]:
X_new = res_krr[cols].values
x_tt_new = ss_krr[cols].values

In [15]:
res_lr = train_targets.copy()
ss_lr.loc[:, train_targets.columns] = 0
res_lr.loc[:, train_targets.columns] = 0

for tar in tqdm(range(train_targets.shape[1])):

#     start_time = time()
    targets = train_targets.values[:, tar]

    if targets.sum() >= N_SPLITS:

        skf = StratifiedKFold(n_splits = N_SPLITS, random_state = 0, shuffle = True)

        for n, (tr, te) in enumerate(skf.split(targets, targets)):

            x_tr, x_val = X_new[tr, tar].reshape(-1, 1), X_new[te, tar].reshape(-1, 1)
            y_tr, y_val = targets[tr], targets[te]

            model = LogisticRegression(penalty = 'none', max_iter = 2000)
            model.fit(x_tr, y_tr)
            ss_lr.loc[:, train_targets.columns[tar]] += model.predict_proba(x_tt_new[:, tar].reshape(-1, 1))[:, 1] / N_SPLITS
            res_lr.loc[te, train_targets.columns[tar]] += model.predict_proba(x_val)[:, 1]

    score = log_loss(train_targets.loc[:, train_targets.columns[tar]].values, res_lr.loc[:, train_targets.columns[tar]].values)
#     print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] LR Target {tar}:', score)

In [16]:
print(f'LR OOF Metric: {log_loss_metric(train_targets.values, res_lr.values)}')

LR OOF Metric: 0.017645709122293074


In [17]:
np.save('klr_oof.npy', res_lr[cols].values)
np.save('klr_sub.npy', ss_lr[cols].values)

# Submit

In [18]:
ss_lr.loc[test['cp_type'] == 1, train_targets.columns] = 0
ss_lr.to_csv('submission.csv', index = False)